# Preparing

## Imports and Functions

In [1]:
import os
import random
import numpy as np
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import regex as re
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models, Sequential, layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, Dropout, LayerNormalization
from tensorflow.keras.callbacks import EarlyStopping

### Normal

In [2]:
folder_path = '/Users/Felix/Google Drive/Felix/code/AgeRecognition/raw_data/Faces'
#folder_path = '/content/UTKFaces/UTKFaces/Faces'

In [3]:
def load_images_from_folder(folder_path,percent=100):
    filenames = [f for f in os.listdir(folder_path) if not f.startswith('.')]
    filenames.sort(key=lambda f: int(re.sub('\D', '', f)))
    images = []
    for filename in filenames:
        img = cv2.imread(os.path.join(folder_path,filename),cv2.IMREAD_UNCHANGED)
        
        scale_percent = percent # percent of original size
        width = int(img.shape[1] * scale_percent / 100)
        height = int(img.shape[0] * scale_percent / 100)
        dim = (width, height)
        
        # resize image
        resized = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)
        cv2.imshow("Resized image", resized)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()
        if img is not None:
            img_colors = cv2.cvtColor(resized, cv2.COLOR_BGR2RGB)
            images.append(img_colors)
    return images

def show_all_images_from_list(img_list,img_df):
    n=int(len(img_list)/5) +1 
    f = plt.figure(figsize=(25,n*5))
    for i,img in enumerate(img_list):
        f.add_subplot(n, 5, i + 1)
        plt.title(f'{img_df.iloc[i]}',
                  fontweight="bold",
                  fontsize=15)
        plt.imshow(img)
        f.tight_layout()

def load_image_data_into_dataframe(folderpath):

    filenames = [f for f in os.listdir(folder_path) if not f.startswith('.')]
    filenames.sort(key=lambda f: int(re.sub('\D', '', f)))
    df = pd.DataFrame()
    for filename in filenames:
        if filename.startswith('81'):
            break
        parsed_name = filename.split('_')
        df= df.append({'age' : int(parsed_name[0]),
                       'gender' : parsed_name[1],
                       'race' : parsed_name[2]}, ignore_index=True)
    df['gender'] = df['gender'].map({'0': 'male', '1': 'female'})
    df['race'] = df['race'].map({'0': 'White', '1': 'Black', '2': 'Asian', '3': 'Indian', '4': 'Others'})
    return df

In [4]:
img_pic=load_images_from_folder(folder_path)

In [5]:
img_df = load_image_data_into_dataframe(folder_path)

## Targets into Age Bins of 5 (Baseline: 6%)

In [6]:
bins= [5*i for i in range(17)]
bins

[0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80]

In [7]:
bins= [5*i for i in range(17)]
labels = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
img_df['age_group'] = pd.cut(img_df['age'], bins=bins, labels=labels)

In [8]:
img_df.tail(305)

,age,gender,race,age_group
21700,75.0,female,Black,14
21701,75.0,female,Asian,14
21702,75.0,female,Asian,14
21703,75.0,female,Indian,14
21704,76.0,male,White,15
...,...,...,...,...
22000,80.0,female,Black,15
22001,80.0,female,Asian,15
22002,80.0,female,Asian,15
22003,80.0,female,Indian,15


In [9]:
img_df['age_group'].value_counts().sort_values()

15     301
14     359
13     463
2      583
12     674
1      816
11     944
3      971
9      971
8     1062
10    1176
7     1825
0     2308
6     2341
4     2545
5     4666
Name: age_group, dtype: int64

## CNN 1 Preperations (Size: Original)

In [10]:
X = img_pic
y = img_df['age_group']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

X_train = np.array(X_train)
X_test =np.array(X_test)
y_train = np.array(y_train)
y_test =np.array(y_test)
X_train.shape

(19804, 200, 200, 3)

In [11]:
X_train = X_train/255 - 0.5
X_test = X_test/255 - 0.5
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

In [12]:
#np.save('X_train', X_train)
#np.save('X_test', X_test)
#np.save('y_train_cat', y_train_cat)
#np.save('y_test_cat', y_test_cat)

In [13]:
print(X_train.shape)
print(X_test.shape)

(19804, 200, 200, 3)
(2201, 200, 200, 3)


## CNN 1 (Accuracy: 29,8 %)

In [14]:
def initialize_model():
    model = models.Sequential()
    ### First convolution & max-pooling
    model.add(layers.Conv2D(8, (4,4), input_shape=(200, 200, 3),
                            activation='relu'))
    model.add(layers.MaxPool2D(pool_size=(2,2)))

    ### Flattening
    model.add(layers.Flatten())

    ### One fully connected
    model.add(layers.Dense(10, activation='relu'))

    ### Last layer (let's say a classification with 10 output)
    model.add(layers.Dense(16, activation='softmax'))
    
    ### Model compilation
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    return model

In [ ]:
model = initialize_model()

es = EarlyStopping(patience=2,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=10, 
          batch_size=32, 
          verbose=1)

In [ ]:
print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

## CNN Preperations Image Size: 100*100

In [ ]:
img_pic=load_images_from_folder(folder_path,50)

In [ ]:
X = img_pic
y = img_df['age_group']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

X_train = np.array(X_train)
X_test =np.array(X_test)
y_train = np.array(y_train)
y_test =np.array(y_test)
X_train.shape

In [ ]:
X_train = X_train/255 - 0.5
X_test = X_test/255 - 0.5
y_train_cat = to_categorical(y_train)
y_test_cat = to_categorical(y_test)

In [ ]:
print(X_train.shape)
print(X_test.shape)

# CNN A

## CNN 2 (1 hidden with 10) 35,7%

In [26]:
def initialize_model():
    model = models.Sequential()
    ### First convolution & max-pooling
    model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),
                            activation='relu'))
    model.add(layers.MaxPool2D(pool_size=(2,2)))

    ### Flattening
    model.add(layers.Flatten())

    ### One fully connected
    model.add(layers.Dense(10, activation='relu'))

    ### Last layer (let's say a classification with 10 output)
    model.add(layers.Dense(16, activation='softmax'))
    
    ### Model compilation
    model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
    return model

In [27]:
model2 = initialize_model()
print(model.summary())

es = EarlyStopping(patience=5,restore_best_weights=True)

# YOUR CODE HERE
model2.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=10, 
          batch_size=32, 
          verbose=1)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 197, 197, 8)       392       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 98, 98, 8)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 76832)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                768330    
_________________________________________________________________
dense_3 (Dense)              (None, 16)                176       
Total params: 768,898
Trainable params: 768,898
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
434/434 [==============================] - 34s 77ms/step - loss: 2.2767 - accuracy: 0.2719 - val_l

In [28]:
print(model2.evaluate(X_test, y_test_cat, verbose=0)[1])

0.3571104109287262


In [42]:
model2.predict(X_test)[5]

array([3.0347572e-05, 1.6294294e-05, 8.7637331e-05, 1.3797468e-03,
       1.6630489e-02, 1.0185414e-01, 1.7867193e-01, 2.0872526e-01,
       1.4289308e-01, 1.2883711e-01, 1.3429570e-01, 5.9380196e-02,
       1.7223625e-02, 9.0944525e-03, 4.0032846e-04, 4.7971428e-04],
      dtype=float32)

In [41]:
y_test[:6]

array([ 5,  4, 15,  9,  0, 10])

## CNN 3 (1 hidden with 50 ) 33,6 %

In [43]:
model = models.Sequential()
### First convolution & max-pooling
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),
                        activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))

### Flattening
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(50, activation='relu'))

### Last layer (let's say a classification with 10 output)
model.add(layers.Dense(16, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=5,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=10, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_6 (Dense)              (None, 50)                921650    
_________________________________________________________________
dense_7 (Dense)              (None, 16)                816       
Total params: 922,858
Trainable params: 922,858
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
434/434 [==============================] - 38s 86ms/step - loss: 2.1161 - accuracy: 0.3039 - val_l

## CNN 4 (1 hidden with 100 )  34,1%

In [44]:
model = models.Sequential()
### First convolution & max-pooling
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),
                        activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))

### Flattening
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(100, activation='relu'))

### Last layer (let's say a classification with 10 output)
model.add(layers.Dense(16, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=5,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=10, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_8 (Dense)              (None, 100)               1843300   
_________________________________________________________________
dense_9 (Dense)              (None, 16)                1616      
Total params: 1,845,308
Trainable params: 1,845,308
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
434/434 [==============================] - 36s 82ms/step - loss: 2.1075 - accuracy: 0.3005 - v

## CNN 5 (2 hidden with 100,50 )  34,94%

In [45]:
model = models.Sequential()
### First convolution & max-pooling
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),
                        activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))

### Flattening
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(100, activation='relu'))

### Second fully connected
model.add(layers.Dense(50, activation='relu'))

### Last layer (let's say a classification with 10 output)
model.add(layers.Dense(16, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])

es = EarlyStopping(patience=5,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=10, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Epoch 1/10
434/434 [==============================] - 29s 66ms/step - loss: 2.1312 - accuracy: 0.2988 - val_loss: 1.9583 - val_accuracy: 0.3167
Epoch 2/10
434/434 [==============================] - 26s 61ms/step - loss: 1.7922 - accuracy: 0.3606 - val_loss: 1.8258 - val_accuracy: 0.3462
Epoch 3/10
434/434 [==============================] - 27s 61ms/step - loss: 1.6449 - accuracy: 0.4015 - val_loss: 1.7820 - val_accuracy: 0.3575
Epoch 4/10
434/434 [==============================] - 25s 58ms/step - loss: 1.5320 - accuracy: 0.4354 - val_loss: 1.7728 - val_accuracy: 0.3521
Epoch 5/10
434/434 [==============================] - 25s 58ms/step - loss: 1.4243 - accuracy: 0.4735 - val_loss: 1.8733 - val_accuracy: 0.3509
Epoch 6/10
434/434 [==============================] - 26s 61ms/step - loss: 1.3152 - accuracy: 0.5130 - val_loss: 1.9021 - val_accuracy: 0.3490
Epoch 7/10
434/434 [==============================] - 26s 59ms/step - loss: 1.1920 - accuracy: 0.5553 - val_loss: 1.9568 - val_accuracy:

## CNN 6 (2 hidden with 100,100 ) 34,3%

In [46]:
model = models.Sequential()
### First convolution & max-pooling
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),
                        activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))

### Flattening
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(100, activation='relu'))

### Second fully connected
model.add(layers.Dense(100, activation='relu'))

### Last layer (let's say a classification with 10 output)
model.add(layers.Dense(16, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])

es = EarlyStopping(patience=5,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=10, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Epoch 1/10
434/434 [==============================] - 30s 67ms/step - loss: 2.0320 - accuracy: 0.3155 - val_loss: 1.8347 - val_accuracy: 0.3470
Epoch 2/10
434/434 [==============================] - 28s 64ms/step - loss: 1.7145 - accuracy: 0.3788 - val_loss: 1.8049 - val_accuracy: 0.3549
Epoch 3/10
434/434 [==============================] - 28s 64ms/step - loss: 1.5892 - accuracy: 0.4150 - val_loss: 1.7649 - val_accuracy: 0.3517
Epoch 4/10
434/434 [==============================] - 27s 62ms/step - loss: 1.4658 - accuracy: 0.4591 - val_loss: 1.7736 - val_accuracy: 0.3470
Epoch 5/10
434/434 [==============================] - 29s 66ms/step - loss: 1.3481 - accuracy: 0.5020 - val_loss: 1.8986 - val_accuracy: 0.3564
Epoch 6/10
434/434 [==============================] - 28s 64ms/step - loss: 1.2186 - accuracy: 0.5517 - val_loss: 1.9102 - val_accuracy: 0.3437
Epoch 7/10
434/434 [==============================] - 26s 60ms/step - loss: 1.0664 - accuracy: 0.6078 - val_loss: 2.0424 - val_accuracy:

## CNN 7 (3 hidden with 50,100,50 )  36,4%

In [47]:
model = models.Sequential()
### First convolution & max-pooling
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),
                        activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))

### Flattening
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(50, activation='relu'))

### Second fully connected
model.add(layers.Dense(100, activation='relu'))

### Third fully connected
model.add(layers.Dense(50, activation='relu'))

### Last layer (let's say a classification with 10 output)
model.add(layers.Dense(16, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=5,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=10, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_16 (Dense)             (None, 50)                921650    
_________________________________________________________________
dense_17 (Dense)             (None, 100)               5100      
_________________________________________________________________
dense_18 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_19 (Dense)             (None, 16)               

## CNN 8 (1 hidden with 300 )  34,2%

In [48]:
model = models.Sequential()
### First convolution & max-pooling
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),
                        activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))

### Flattening
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(300, activation='relu'))

### Last layer (let's say a classification with 10 output)
model.add(layers.Dense(16, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])

print(model.summary())

es = EarlyStopping(patience=5,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=10, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_20 (Dense)             (None, 300)               5529900   
_________________________________________________________________
dense_21 (Dense)             (None, 16)                4816      
Total params: 5,535,108
Trainable params: 5,535,108
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
434/434 [==============================] - 38s 87ms/step - loss: 2.0260 - accuracy: 0.3130 - v

## CNN 9 (2 hidden with 100,20 )  35,3%

In [49]:
model = models.Sequential()
### First convolution & max-pooling
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),
                        activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))

### Flattening
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(100, activation='relu'))

### Second fully connected
model.add(layers.Dense(20, activation='relu'))

### Last layer (let's say a classification with 10 output)
model.add(layers.Dense(16, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=5,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=10, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 18432)             0         
_________________________________________________________________
dense_22 (Dense)             (None, 100)               1843300   
_________________________________________________________________
dense_23 (Dense)             (None, 20)                2020      
_________________________________________________________________
dense_24 (Dense)             (None, 16)                336       
Total params: 1,846,048
Trainable params: 1,846,048
Non-trainable params: 0
____________________________________________

## CNN 10 (3 hidden with 20, 50, 20 )  34,76%

In [50]:
model = models.Sequential()
### First convolution & max-pooling
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),
                        activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))

### Flattening
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(20, activation='relu'))

### Second fully connected
model.add(layers.Dense(50, activation='relu'))

### Third fully connected
model.add(layers.Dense(20, activation='relu'))

### Last layer (let's say a classification with 10 output)
model.add(layers.Dense(16, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=5,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=10, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 18432)             0         
_________________________________________________________________
dense_25 (Dense)             (None, 20)                368660    
_________________________________________________________________
dense_26 (Dense)             (None, 50)                1050      
_________________________________________________________________
dense_27 (Dense)             (None, 20)                1020      
_________________________________________________________________
dense_28 (Dense)             (None, 16)              

# CNN B

## CNN 1 (3 hidden with 50,100,50 ) - (Epochs:100, Patience: 10)   35,58%
**High_Acc - low Val_Acc**

In [53]:
model = models.Sequential()
### First convolution & max-pooling & Flattening
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(50, activation='relu'))

### Second fully connected
model.add(layers.Dense(100, activation='relu'))

### Third fully connected
model.add(layers.Dense(50, activation='relu'))

model.add(layers.Dense(16, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=5,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=100, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_13 (Conv2D)           (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_13 (Flatten)         (None, 18432)             0         
_________________________________________________________________
dense_34 (Dense)             (None, 50)                921650    
_________________________________________________________________
dense_35 (Dense)             (None, 100)               5100      
_________________________________________________________________
dense_36 (Dense)             (None, 50)                5050      
_________________________________________________________________
dense_37 (Dense)             (None, 16)              

## CNN 1.2 - Dropout, Hidden Layer 50% => 30,7%

In [57]:
model = models.Sequential()
### First convolution & max-pooling & Flattening
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Flatten())


### One fully connected
model.add(layers.Dense(50, activation='relu'))
model.add(Dropout(0.5))

### Second fully connected
model.add(layers.Dense(100, activation='relu'))
model.add(Dropout(0.5))

### Third fully connected
model.add(layers.Dense(50, activation='relu'))
model.add(Dropout(0.5))

model.add(layers.Dense(16, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=10,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=100, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_17 (Flatten)         (None, 18432)             0         
_________________________________________________________________
dense_46 (Dense)             (None, 50)                921650    
_________________________________________________________________
dropout_11 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_47 (Dense)             (None, 100)               5100      
_________________________________________________________________
dropout_12 (Dropout)         (None, 100)             

## CNN 1.3 Dropout First and Hidden Layer 20%    34,2 %

In [58]:
model = models.Sequential()
### First convolution & max-pooling & Flattening
model.add(Dropout(0.2, input_shape=(100,100,3,)))
model.add(layers.Conv2D(8, (4,4), activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(50, activation='relu'))
model.add(Dropout(0.2))

### Second fully connected
model.add(layers.Dense(100, activation='relu'))
model.add(Dropout(0.2))

### Third fully connected
model.add(layers.Dense(50, activation='relu'))
model.add(Dropout(0.2))

### Last layer 
model.add(layers.Dense(16, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=10,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=100, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_14 (Dropout)         (None, 100, 100, 3)       0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_18 (Flatten)         (None, 18432)             0         
_________________________________________________________________
dense_50 (Dense)             (None, 50)                921650    
_________________________________________________________________
dropout_15 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_51 (Dense)             (None, 100)             

## CNN 1.4 Dropout, Hidden Layer 40% => 33,8%

In [59]:
model = models.Sequential()
### First convolution & max-pooling & Flattening
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(50, activation='relu'))
model.add(Dropout(0.4))

### Second fully connected
model.add(layers.Dense(100, activation='relu'))
model.add(Dropout(0.4))

### Third fully connected
model.add(layers.Dense(50, activation='relu'))
model.add(Dropout(0.4))

### Last layer 
model.add(layers.Dense(16, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=10,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=100, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_23 (Conv2D)           (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_19 (Flatten)         (None, 18432)             0         
_________________________________________________________________
dense_54 (Dense)             (None, 50)                921650    
_________________________________________________________________
dropout_18 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_55 (Dense)             (None, 100)               5100      
_________________________________________________________________
dropout_19 (Dropout)         (None, 100)             

## CNN 1.5 Dropout, Hidden Layer 30% => 35,1%

In [60]:
model = models.Sequential()
### First convolution & max-pooling & Flattening
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(50, activation='relu'))
model.add(Dropout(0.3))

### Second fully connected
model.add(layers.Dense(100, activation='relu'))
model.add(Dropout(0.3))

### Third fully connected
model.add(layers.Dense(50, activation='relu'))
model.add(Dropout(0.3))

### Last layer 
model.add(layers.Dense(16, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=10,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=100, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_24 (Conv2D)           (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_24 (MaxPooling (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_20 (Flatten)         (None, 18432)             0         
_________________________________________________________________
dense_58 (Dense)             (None, 50)                921650    
_________________________________________________________________
dropout_21 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_59 (Dense)             (None, 100)               5100      
_________________________________________________________________
dropout_22 (Dropout)         (None, 100)             

## CNN 1.6 Dropout, Hidden Layer 20% => 34,98%

In [61]:
model = models.Sequential()
### First convolution & max-pooling & Flattening
model.add(layers.Conv2D(8, (4,4), input_shape=(100, 100, 3),activation='relu'))
model.add(layers.MaxPool2D(pool_size=(2,2)))
model.add(layers.Flatten())

### One fully connected
model.add(layers.Dense(50, activation='relu'))
model.add(Dropout(0.2))

### Second fully connected
model.add(layers.Dense(100, activation='relu'))
model.add(Dropout(0.2))

### Third fully connected
model.add(layers.Dense(50, activation='relu'))
model.add(Dropout(0.2))

### Last layer 
model.add(layers.Dense(16, activation='softmax'))

### Model compilation
model.compile(loss='categorical_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])
print(model.summary())

es = EarlyStopping(patience=10,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=100, 
          batch_size=32, 
          verbose=1)

print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 97, 97, 8)         392       
_________________________________________________________________
max_pooling2d_25 (MaxPooling (None, 48, 48, 8)         0         
_________________________________________________________________
flatten_21 (Flatten)         (None, 18432)             0         
_________________________________________________________________
dense_62 (Dense)             (None, 50)                921650    
_________________________________________________________________
dropout_24 (Dropout)         (None, 50)                0         
_________________________________________________________________
dense_63 (Dense)             (None, 100)               5100      
_________________________________________________________________
dropout_25 (Dropout)         (None, 100)             

# CNN C Something new (Copy from Cifar Classification) 37,6 %

In [ ]:
model = models.Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(100, 100, 3)))
#model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
#model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(128, (2, 2), activation='relu', padding='same'))
#model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Dropout(0.3))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(16, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

print(model.summary())

es = EarlyStopping(patience=10,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=100, 
          batch_size=32, 
          verbose=1)

In [ ]:
print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

# CNN D Copy from Internet   36,7 %

In [76]:
model = Sequential()
model.add(Conv2D(input_shape=(100, 100, 3), filters=96, kernel_size=(7, 7), strides=(2, 2), padding='valid', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(LayerNormalization())
model.add(Conv2D(filters=256, kernel_size=(5, 5), strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(LayerNormalization())
model.add(Conv2D(filters=256, kernel_size=(3, 3), strides=1, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
model.add(LayerNormalization())

model.add(Flatten())
model.add(Dense(units=512, activation='relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(units=512, activation='relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(units=16, activation='softmax'))

print(model.summary())

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

es = EarlyStopping(patience=10,restore_best_weights=True)

# YOUR CODE HERE
model.fit(X_train, y_train_cat, 
          validation_split=0.3,
          callbacks=[es],
          epochs=100, 
          batch_size=32, 
          verbose=1)

Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_33 (Conv2D)           (None, 47, 47, 96)        14208     
_________________________________________________________________
max_pooling2d_33 (MaxPooling (None, 23, 23, 96)        0         
_________________________________________________________________
layer_normalization_3 (Layer (None, 23, 23, 96)        192       
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 23, 23, 256)       614656    
_________________________________________________________________
max_pooling2d_34 (MaxPooling (None, 11, 11, 256)       0         
_________________________________________________________________
layer_normalization_4 (Layer (None, 11, 11, 256)       512       
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 11, 11, 256)     

In [77]:
print(model.evaluate(X_test, y_test_cat, verbose=0)[1])

0.3671058714389801


# Evaluation of the Best Model

In [ ]:
best_model = model